The first step of preprocessing for the data we are using in class will be cleaning it for initial visualization. The comment bodies will need escape sequences removed, emojis/invalid characters parsed and removed, and any other issues in our data that could prevent a seamless exploration.

In [71]:
#package and data importing and loading

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

subreddits = ["kanye", "askScience", "PoliticalDiscussion", "socialism"]

#comments are pulled from the top 40 posts from the past month in each subreddit.
dataframes = []
for sub in subreddits:
    df = pd.read_csv(f"data/comments_{sub}.csv")
    dataframes.append(df)

In [72]:
print(dataframes[0].columns)
for i in range(len(dataframes)):
    print(subreddits[i])
    print(dataframes[i].shape)

Index(['postTag', 'user', 'comment_score', 'comment_body', 'mod_deleted',
       'user_deleted', 'verified', 'is_gold', 'has_verified_email',
       'link_karma', 'total_karma', 'created_utc', 'comment_karma'],
      dtype='object')
kanye
(42654, 13)
askScience
(11134, 13)
PoliticalDiscussion
(31235, 13)
socialism
(3038, 13)


Drop comments that were removed too quickly and so were not archived. Also drop comments automatically generated by mods.

In [73]:
mods = ["https://www.reddit.com/user/AutoModerator", "https://www.reddit.com/user/socialism-ModTeam"]
for i in range(len(dataframes)):
    dataframes[i] = dataframes[i].dropna(subset=["user"])
    dataframes[i] = dataframes[i][~dataframes[i]["user"].isin(mods)]
    print(subreddits[i])
    print(dataframes[i].shape)

kanye
(40436, 13)
askScience
(6679, 13)
PoliticalDiscussion
(28026, 13)
socialism
(2620, 13)


The distribution of the target is very uneven so we need to make sure we don't overfit on the majority class.


In [32]:
for i in range(len(dataframes)):
    print(subreddits[i])
    print(dataframes[i].groupby(["mod_deleted"]).size())
    print(dataframes[i].groupby(["user_deleted"]).size())
    print()

kanye
mod_deleted
0    8243
1     141
dtype: int64
user_deleted
0    8147
1     237
dtype: int64

askScience
mod_deleted
0    5146
1    1533
dtype: int64
user_deleted
0    6570
1     109
dtype: int64

PoliticalDiscussion
mod_deleted
0    27239
1      837
dtype: int64
user_deleted
0    27411
1      665
dtype: int64

socialism
mod_deleted
0.0    2487
1.0     229
dtype: int64
user_deleted
0.0    2655
1.0      61
dtype: int64



In [ ]:
#cleaning escape sequences, invalid words, deleted comments, and other things that won't serve to help our analysis. regex?

Our analysis will be looking at which variables (from the data we collected) are the most useful in classifying whether a comment gets deleted, and if so, whether the user deleted it themselves, or a moderator deleted it. Can we predict based on certain keywords, or a threshold for karma, or any other classifers, what the outcome of the comments status will be? Could this information we use be utilized to enhance the auto moderator currently used on reddit?

The main classifier/variable we are studying will obviously be the comment bodies, as that content will be most critical to parsing the synoposis of messages that routinely get deleted or not. Thus, the data will be mostly free text, with no predefined features. As such, we will use multiple techniques to create training data to be used in model selection and training. Correlations discovered between account creation, comment karma, will be observed but will require less cleaning.

In [75]:
#CountVectorizer 
#We will use CountVectorizer during vectorization of datasets.
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

comments_trans = []

# Added stemming to vectorization but may not be necessary, max_df has a bigger impact
porter = SnowballStemmer("english")
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

for i in range(len(dataframes)):
    comments = dataframes[i]["comment_body"].apply(lambda x: " ".join(tokenizer_porter(x)))
    vect = CountVectorizer(stop_words=stopwords.words('english'),max_df=.5, ngram_range=(1,2) ,max_features=50, strip_accents="unicode")
    vect.fit(comments)
    print(subreddits[i])
    print(vect.get_feature_names_out())
    print()
    comments_trans.append(vect.transform(comments).toarray())
    vect = CountVectorizer(stop_words=stopwords.words('english'),max_df=.5, ngram_range=(2,2) ,max_features=25, strip_accents="unicode")
    vect.fit(comments)
    print(vect.get_feature_names_out())
    print()


kanye
['actual' 'also' 'ani' 'anti' 'becaus' 'black' 'call' 'even' 'fuck' 'get'
 'go' 'good' 'hate' 'jew' 'jewish' 'kany' 'know' 'like' 'lol' 'make' 'man'
 'mean' 'music' 'need' 'one' 'onli' 'peopl' 'person' 'point' 'realli'
 'right' 'said' 'say' 'see' 'shit' 'someon' 'still' 'take' 'talk' 'thing'
 'think' 'time' 'tri' 'use' 'want' 'way' 'whi' 'white' 'would' 'ye']

['act like' 'anti semit' 'black peopl' 'black people' 'eric andr'
 'feel like' 'georg floyd' 'get help' 'https www' 'jewish peopl'
 'jewish people' 'kany said' 'kany west' 'like kany' 'look like'
 'mental health' 'mental ill' 'need help' 'peopl like' 'piec shit'
 'seem like' 'social media' 'sound like' 'white peopl' 'year old']

askScience
['actual' 'also' 'ani' 'becaus' 'bodi' 'caus' 'cell' 'could' 'differ'
 'doe' 'earth' 'effect' 'enough' 'even' 'get' 'go' 'human' 'know' 'like'
 'long' 'look' 'lot' 'make' 'mani' 'mean' 'much' 'need' 'one' 'onli'
 'peopl' 'pressur' 'realli' 'say' 'see' 'someth' 'still' 'system' 'take'
 'th

In [10]:
#TF-IDF Rescaling Calculations. -> [Utilizing a param grid or pipeline could simplify this process.]
#-> A statistical measure to evaluate how relevant a word is to a document.
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.pipeline import make_pipeline
kanye_tfidf = make_pipeline(CountVectorizer(stop_words=stopwords.words('english'), max_features=10), TfidfTransformer()).fit_transform(kanyeData['comment_body'].apply(lambda x: " ".join(x)))
kanye_tfidf.toarray()

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.76848418],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [12]:
#N-grams
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

param_grid = {"countvectorizer__ngram_range":[(1, 2), (2, 5)],
              "countvectorizer__min_df": [2, 3]
             }

grid = GridSearchCV(make_pipeline(CountVectorizer(analyzer="char"), LogisticRegression()), param_grid=param_grid,
                                  cv=10, scoring="f1_macro", return_train_score=True)